# Exploration des données Velib - API JCDecaux

**Rôle 2 - Data Engineer Batch**

Objectif : Explorer les données Velib récupérées depuis l'API JCDecaux avant le traitement batch.

In [1]:
from pyspark.sql import SparkSession
from pyspark.sql.functions import col, count, avg, min, max, sum, desc
import pandas as pd
import matplotlib.pyplot as plt
import requests
import json

## 2. Initialisation de Spark (local)

In [2]:
spark = SparkSession.builder \
    .appName("VelibExploration") \
    .master("local[*]") \
    .getOrCreate()

print(f"✓ Spark {spark.version} initialisé")

✓ Spark 4.0.1 initialisé


## 3. Récupération des données depuis l'API

In [4]:
# Configuration API
API_KEY = "0a2ee2814ab033c2a34c4ae92df2c9b0cf1a5471"

# Récupérer toutes les stations depuis l'API
url = f"https://api.jcdecaux.com/vls/v3/stations?apiKey={API_KEY}"
response = requests.get(url, timeout=30)
stations_data = response.json()

print(f"✓ {len(stations_data)} stations récupérées depuis l'API")

# Sauvegarder temporairement en fichier JSON
import tempfile
import os

temp_file = os.path.join(tempfile.gettempdir(), "velib_temp.json")
with open(temp_file, 'w', encoding='utf-8') as f:
    json.dump(stations_data, f)

# Créer un DataFrame Spark depuis le fichier JSON
df = spark.read.json(temp_file)

print(f"✓ DataFrame créé avec {df.count()} lignes")

✓ 2455 stations récupérées depuis l'API
✓ DataFrame créé avec 2455 lignes
✓ DataFrame créé avec 2455 lignes


## 4. Explorer les colonnes

In [ ]:
# Afficher le schéma (structure JSON de l'API)
df.printSchema()

In [6]:
# Aperçu des premières lignes
df.show(5, truncate=False)

+---------------------------------------------+-------+-----+---------+------------+--------------------+---------------------------+----------------------------------+------+--------+--------------+----------------------+-----+------+---------------------------+
|address                                      |banking|bonus|connected|contractName|lastUpdate          |mainStands                 |name                              |number|overflow|overflowStands|position              |shape|status|totalStands                |
+---------------------------------------------+-------+-----+---------+------------+--------------------+---------------------------+----------------------------------+------+--------+--------------+----------------------+-----+------+---------------------------+
|2 RUE GATIEN ARNOULT                         |false  |false|true     |toulouse    |2025-12-11T08:46:17Z|{{15, 11, 11, 0, 4, 0}, 15}|00055 - ST-SERNIN - GATIEN-ARNOULT|55    |false   |NULL          |{43.60902

In [7]:
# Liste des colonnes de l'API
print("Colonnes disponibles :")
for col_name in df.columns:
    print(f"  - {col_name}")

Colonnes disponibles :
  - address
  - banking
  - bonus
  - connected
  - contractName
  - lastUpdate
  - mainStands
  - name
  - number
  - overflow
  - overflowStands
  - position
  - shape
  - status
  - totalStands


## 5. Vérifier les valeurs nulles

In [8]:
from pyspark.sql.functions import isnan, when, count as spark_count

# Compter les valeurs nulles par colonne
null_counts = df.select([spark_count(when(col(c).isNull(), c)).alias(c) for c in df.columns])
null_counts.show()

+-------+-------+-----+---------+------------+----------+----------+----+------+--------+--------------+--------+-----+------+-----------+
|address|banking|bonus|connected|contractName|lastUpdate|mainStands|name|number|overflow|overflowStands|position|shape|status|totalStands|
+-------+-------+-----+---------+------------+----------+----------+----+------+--------+--------------+--------+-----+------+-----------+
|      0|      0|    0|        0|           0|         2|         0|   0|     0|       0|          2455|       0| 2455|     0|          0|
+-------+-------+-----+---------+------------+----------+----------+----+------+--------+--------------+--------+-----+------+-----------+



## 6. Statistiques descriptives

In [9]:
# Statistiques basiques
df.describe().show()

+-------+--------------------+------------+--------------------+----------------+------------------+--------------+-----+------+
|summary|             address|contractName|          lastUpdate|            name|            number|overflowStands|shape|status|
+-------+--------------------+------------+--------------------+----------------+------------------+--------------+-----+------+
|  count|                2455|        2455|                2453|            2455|              2455|             0|    0|  2455|
|   mean|                NULL|        NULL|                NULL|            NULL|1489.4338085539714|          NULL| NULL|  NULL|
| stddev|                NULL|        NULL|                NULL|            NULL|3906.9104852725127|          NULL| NULL|  NULL|
|    min|                    |      amiens|2022-08-30T15:30:02Z|#00001-LEON XIII|                 1|          NULL| NULL|CLOSED|
|    max|西町３番 3 Nishi-cho|     vilnius|2025-12-11T08:46:41Z|  西町 NISHI-CHO|             65009|   

## 7. Extraire et analyser la capacité des stations

In [10]:
# Extraire les données imbriquées et analyser la capacité
df_clean = df.select(
    col("contractName").alias("contract"),
    col("name").alias("station_name"),
    col("totalStands.capacity").alias("capacity")
).filter(col("capacity").isNotNull())

# Capacité par station
capacity_by_station = df_clean.groupBy("contract", "station_name") \
    .agg(max("capacity").alias("capacity_total")) \
    .orderBy(desc("capacity_total"))

capacity_by_station.show(10, truncate=False)

+---------+------------------------------------------+--------------+
|contract |station_name                              |capacity_total|
+---------+------------------------------------------+--------------+
|nantes   |070-GARE DE NANTES SUD                    |70            |
|lyon     |10002 - INSA                              |55            |
|amiens   |138 - CITADELLE                           |50            |
|lyon     |2018 - HOTEL DIEU / PONT DE LA GUILLOTIERE|50            |
|lyon     |10006 - CHARPENNES                        |46            |
|toyama   |富山大学前 TOYAMA UNIVERSITY MAE          |45            |
|bruxelles|179 - CELTES / KELTEN                     |44            |
|bruxelles|100 - LUXEMBOURG / LUXEMBURG              |44            |
|toulouse |00010 - PLACE ESQUIROL                    |44            |
|bruxelles|058 - GARE DU NORD / NOORDSTATION         |43            |
+---------+------------------------------------------+--------------+
only showing top 10 rows


## 8. Statistiques globales

In [11]:
# Nombre de stations
nb_stations = df_clean.select("station_name").distinct().count()

# Nombre de contrats
nb_contracts = df_clean.select("contract").distinct().count()

# Capacité totale
total_capacity = df_clean.agg(sum("capacity")).collect()[0][0]

# Capacité moyenne
avg_capacity = df_clean.agg(avg("capacity")).collect()[0][0]

print(f"Nombre de contrats : {nb_contracts}")
print(f"Nombre de stations : {nb_stations}")
print(f"Capacité totale : {total_capacity}")
print(f"Capacité moyenne : {avg_capacity:.2f}")

Nombre de contrats : 19
Nombre de stations : 2455
Capacité totale : 50511
Capacité moyenne : 20.57


## 9. Visualisation (optionnel)

In [ ]:
# Convertir en Pandas pour visualisation
top_stations = capacity_by_station.limit(15).toPandas()

plt.figure(figsize=(12, 6))
plt.barh(top_stations['station_name'], top_stations['capacity_total'])
plt.xlabel('Capacité')
plt.ylabel('Station')
plt.title('Top 15 des stations par capacité')
plt.gca().invert_yaxis()
plt.tight_layout()
plt.show()